In [1]:
from azureml.core import Workspace, Experiment, Datastore
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

ws = Workspace.from_config()
blob_store = Datastore(ws, "workspaceblobstore")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, 
      'Blog storage:' + blob_store.name,
      sep = '\n')

exp = Experiment(workspace=ws, name="Exp_Proj01")
run = exp.start_logging()

Workspace name: udc-machinelearning01
Azure region: eastus2
Subscription id: 7023dccf-8c31-41db-809e-9f442490bfe7
Resource group: rg-udc-ml-dev-eastus-01
Blog storage:workspaceblobstore


SnapshotException: SnapshotException:
	Message: Failed to take a snapshot of .. Pass snapshot_directory=None to start_logging to skip taking a snapshot of the given directory.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Failed to take a snapshot of .. Pass snapshot_directory=None to start_logging to skip taking a snapshot of the given directory."
    }
}

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Personal Subcription - using existing cluster
cluster_name = "udc-ml-cluster-01"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster.')
except ComputeTargetException:
    print('Creating a new cluster.')
    # 4 cores, 8GB RAM, 32GB storage
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_F4s_v2',
                                                          max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)


In [2]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C" :        choice(0.1,1,2,5,10),
        "--max_iter" : choice(10,100,150,200,250)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory= './training',
    script="train.py",
    compute_target=cluster,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling = ps, 
    policy = policy,
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 15,
    max_concurrent_runs = 4,
    run_config = src
)

NameError: name 'cluster' is not defined

In [15]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdrive_run = exp.submit(hyperdrive_config)
RunDetails(hdrive_run).show()
hdrive_run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_471a09b1-a2ab-466e-a994-d395638b1c58',
 'target': 'udc-ml-cluster-01',
 'status': 'Completed',
 'startTimeUtc': '2024-03-13T03:57:37.016699Z',
 'endTimeUtc': '2024-03-13T04:08:12.328874Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e713b0d6-492c-4345-ba71-451df99e3fa0',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '25',
  'best_child_run_id': 'HD_471a09b1-a2ab-466e-a994-d395638b1c58_1',
  'score': '0.9156553398058253',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_471a09b1-a2ab-466e-a994-d395638b1c58_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlC

In [16]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# current primary metric is accuracy. Or we can use experiment.get_best_run(metric='accuracy')
best_run = hdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())

{'Regularization Strength:': 1.0, 'Max iterations:': 250, 'Accuracy': 0.9156553398058253}


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
datapath = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
     # default behavior: support_multi_line=False
ds = TabularDatasetFactory.from_delimited_files(path=datapath)

In [4]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [5]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=40,
    task="classification",
    enable_early_stopping = True,
    training_data= ds,
    label_column_name="y",
    primary_metric= "accuracy",
    n_cross_validations= 5)

In [6]:
# Submit your automl run

### YOUR CODE HERE ###
exp2 = Experiment(workspace=ws, name="Exp_Proj01_AutoML")
#run = exp2.start_logging()
automl_run = exp2.submit(config=automl_config, show_output=True)
automl_run.wait_for_completion()

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
Exp_Proj01_AutoML,AutoML_465dbc08-06d7-4492-a36e-6253c50d86c4,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


2024/03/13 04:38:01 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |yes                             |32950                                 |
+------------------------------+--------------------------------+-----

2024-03-13:05:12:28,831 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-03-13:05:12:34,743 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-03-13:05:12:34,745 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-03-13:05:12:40,449 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-03-13:05:12:40,451 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and ma

StackEnsemble                                  0:01:22             0.9155    0.9176
Stopping criteria reached at iteration 22. Ending experiment.
********************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrogate model for explanations
Current status: EngineeredFeatureExplanations. Computation of engineered features started


2024-03-13:05:14:47,826 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


{'runId': 'AutoML_465dbc08-06d7-4492-a36e-6253c50d86c4',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2024-03-13T04:29:07.077331Z',
 'endTimeUtc': '2024-03-13T05:13:11.703654Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 40 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"Exp_Proj01_AutoML","subscription_id":"7023dccf-8c31-41db-809e-9f442490bfe7","resource_group":"rg-udc-ml-dev-eastus-01","workspace_name":"udc-machinelearning01","region":"eastus2","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enab

In [7]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_model, model = automl_run.get_output()
print(best_run_model.get_metrics())

{'AUC_weighted': 0.9482968699034895, 'recall_score_macro': 0.7605483372411028, 'average_precision_score_weighted': 0.9561002833983718, 'accuracy': 0.9176024279210926, 'norm_macro_recall': 0.5210966744822055, 'average_precision_score_micro': 0.9817105034948141, 'AUC_micro': 0.9810328473960407, 'average_precision_score_macro': 0.8277495723560543, 'precision_score_macro': 0.8004436835919705, 'weighted_accuracy': 0.9566080416870246, 'balanced_accuracy': 0.7605483372411028, 'f1_score_macro': 0.7784556679861859, 'AUC_macro': 0.9482968699034895, 'log_loss': 0.28399807570987334, 'precision_score_weighted': 0.9128485114392161, 'f1_score_micro': 0.9176024279210926, 'recall_score_weighted': 0.9176024279210926, 'f1_score_weighted': 0.914702066449346, 'recall_score_micro': 0.9176024279210926, 'matthews_correlation': 0.5595409903486122, 'precision_score_micro': 0.9176024279210926, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_465dbc08-06d7-4492-a36e-6253c50d86c4_20/confusion_matrix

In [23]:
# Remove the Compute object from the workspace.
cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

